# Direct Solvers

The purpose of this notebook is to compare commonly available direct solvers from python interface. Here we focus on multi-threaded solvers that can be run on a single node with several cores.
For now, we compare SuperLU, UMFPACK and Intel MKL PARDISO. SuperLU is the default solver in scipy, UMFPACK requires the installation of scikit-umfpack and Intel MKL PARDISO requires PyPardiso.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg as spla

import ibmos as ib

In [3]:
import scipy.linalg as la

# Case initialization
... from [Unsteady flow around cylinder (Re=200)](../1-Basic/121-CylinderRe200.ipynb)

In [4]:
s1 = ib.stretching(256, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
s2 = ib.stretching(128, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
x = np.r_[-s2[::-1], s1[1:]]

s = ib.stretching(192, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
y = np.r_[-s[::-1], s[1:]]

solver = ib.Solver(x, y, iRe=1/200, Co=0.015/0.033)
del x, y, s1, s2

solver.set_solids(ib.shapes.cylinder("cylinder", 0, 0, 0.5, solver.dxmin))

fvBC = (lambda s, t: np.zeros_like(s),)*4
fuBC = (lambda s, t: np.ones_like(s),)*4
fsBC = ((lambda ξ, η, t: np.zeros_like(ξ),)*2, )

u, v, p, *f = solver.reshape(solver.unpack(solver.zero()), p0=0)
c, r, ϵ = -10, 1.0, 0.01
u[:,:] = 1.0-ϵ*solver.fluid.u.y[:,np.newaxis]*np.exp(-((solver.fluid.u.x[np.newaxis,:]-c)**2 + solver.fluid.u.y[:,np.newaxis]**2)/r**2)
v[:,:] = ϵ*(solver.fluid.v.x[np.newaxis,:]-c)*np.exp(-((solver.fluid.v.x[np.newaxis,:]-c)**2 + solver.fluid.v.y[:,np.newaxis]**2)/r**2)
x0 = solver.pack(u, v, p, *f)

In [5]:
n = 10

# SuperLU

In [6]:
solver.set_solver(ib.tools.solver_superlu)

## Without fractional step method

In [7]:
solver.set_fractional_step(False)

In [8]:
%time x1, *_ = solver.steps(x0, fuBC, fvBC, fsBC, number=1, checkSolvers=True)

Initializing solver...done.
       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) 
       1  1.50000e-02  1.39370e+03  8.94021e+04  2.32815e+02 -5.06399e-08  5.70771e-16 
CPU times: user 9min 23s, sys: 15.4 s, total: 9min 39s
Wall time: 29.7 s


In [9]:
%time x, *_ = solver.steps(x1, fuBC, fvBC, fsBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) 
       1  1.50000e-02  3.94895e+02  8.88363e+04  2.11691e+00  4.16393e-08  5.49161e-16 
       2  3.00000e-02  3.90576e+02  1.25370e+03  1.98338e+00  4.32569e-08  5.53198e-16 
       3  4.50000e-02  3.87237e+02  1.20493e+03  1.84122e+00  4.44633e-08  5.46641e-16 
       4  6.00000e-02  3.85533e+02  7.66895e+02  1.72072e+00  4.56212e-08  5.46332e-16 
       5  7.50000e-02  3.84987e+02  2.81708e+02  1.62136e+00  4.67406e-08  5.48750e-16 
       6  9.00000e-02  3.84852e+02  8.42332e+01  1.54062e+00  4.78309e-08  5.47454e-16 
       7  1.05000e-01  3.84604e+02  1.41523e+02  1.47479e+00  4.89012e-08  5.46900e-16 
       8  1.20000e-01  3.84162e+02  2.65593e+02  1.41967e+00  4.99592e-08  5.53951e-16 
       9  1.35000e-01  3.83775e+02  2.56340e+02  1.37208e+00  5.10146e-08  5.49988e-16 
      10  1.50000e-01  3.83613e+02  1.14614e+02  1.33072e+00  5.20720e-08  5.46620e-16 
CPU times: user 1min 32s, sys: 2

## With fractional step method

In [10]:
solver.set_fractional_step(True)

In [11]:
%time x1, *_ = solver.steps(x0, fuBC, fvBC, fsBC, number=1, checkSolvers=True)

Initializing solver...done.
       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  1.39365e+03  8.93986e+04  2.32804e+02 -5.06434e-08  2.08546e-16  8.74836e-15 
CPU times: user 9min 16s, sys: 12.4 s, total: 9min 28s
Wall time: 24.6 s


In [12]:
%time x, *_ = solver.steps(x1, fuBC, fvBC, fsBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  3.94809e+02  8.88307e+04  2.11836e+00  4.16407e-08  2.15450e-16  1.26233e-14 
       2  3.00000e-02  3.90526e+02  1.24713e+03  1.98458e+00  4.32535e-08  2.14544e-16  1.15546e-14 
       3  4.50000e-02  3.87209e+02  1.20003e+03  1.84210e+00  4.44603e-08  2.15958e-16  1.04558e-14 
       4  6.00000e-02  3.85515e+02  7.64226e+02  1.72136e+00  4.56186e-08  2.15692e-16  9.94163e-15 
       5  7.50000e-02  3.84973e+02  2.80624e+02  1.62183e+00  4.67380e-08  2.16870e-16  9.05561e-15 
       6  9.00000e-02  3.84840e+02  8.37754e+01  1.54098e+00  4.78288e-08  2.15673e-16  9.21257e-15 
       7  1.05000e-01  3.84593e+02  1.40824e+02  1.47506e+00  4.88990e-08  2.15249e-16  1.00008e-14 
       8  1.20000e-01  3.84154e+02  2.64729e+02  1.41987e+00  4.99570e-08  2.16055e-16  8.63507e-15 
       9  1.35000e-01  3.83768e+02  2.55747e+02  1.37224e+00  5.10123e-08  2.15919e-16  8.8

# UMFPACK

In [ ]:
solver.set_solver(ib.tools.solver_umfpack)

## Without fracional step method

In [ ]:
solver.set_fractional_step(False)

In [ ]:
%time x1, *_ = solver.steps(x0, fuBC, fvBC, fsBC, number=1, checkSolvers=True)

In [ ]:
%time x, *_ = solver.steps(x1, fuBC, fvBC, fsBC, number=n, checkSolvers=True)

## With fractional step method

In [ ]:
solver.set_fractional_step(True)

In [ ]:
%time x1, *_ = solver.steps(x0, fuBC, fvBC, fsBC, number=1, checkSolvers=True)

In [ ]:
%time x, *_ = solver.steps(x1, fuBC, fvBC, fsBC, number=n, checkSolvers=True)

# PARDISO

In [13]:
solver.set_solver(ib.tools.solver_pardiso)

## Without fractional step method

In [14]:
solver.set_fractional_step(False)

In [15]:
%time x1, *_ = solver.steps(x0, fuBC, fvBC, fsBC, number=1, checkSolvers=True)

Initializing solver...done.
       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) 
       1  1.50000e-02  1.39370e+03  8.94021e+04  2.32815e+02 -5.03008e-08  4.65495e-16 
CPU times: user 26.3 s, sys: 2.18 s, total: 28.4 s
Wall time: 2.26 s


In [16]:
%time x, *_ = solver.steps(x1, fuBC, fvBC, fsBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) 
       1  1.50000e-02  3.94895e+02  8.88363e+04  2.11691e+00  4.13148e-08  1.63379e-16 
       2  3.00000e-02  3.90576e+02  1.25370e+03  1.98338e+00  4.32551e-08  1.63392e-16 
       3  4.50000e-02  3.87237e+02  1.20493e+03  1.84122e+00  4.44612e-08  1.62601e-16 
       4  6.00000e-02  3.85533e+02  7.66895e+02  1.72072e+00  4.56198e-08  1.63173e-16 
       5  7.50000e-02  3.84987e+02  2.81708e+02  1.62136e+00  4.67397e-08  1.62916e-16 
       6  9.00000e-02  3.84852e+02  8.42332e+01  1.54062e+00  4.78309e-08  1.62896e-16 
       7  1.05000e-01  3.84604e+02  1.41523e+02  1.47479e+00  4.89009e-08  1.62934e-16 
       8  1.20000e-01  3.84162e+02  2.65593e+02  1.41967e+00  4.99587e-08  1.62345e-16 
       9  1.35000e-01  3.83775e+02  2.56340e+02  1.37208e+00  5.10140e-08  1.62386e-16 
      10  1.50000e-01  3.83613e+02  1.14614e+02  1.33072e+00  5.20719e-08  1.62711e-16 
CPU times: user 39.9 s, sys: 1.6

## With fractional step method

In [17]:
solver.set_fractional_step(True)

In [18]:
%time x1, *_ = solver.steps(x0, fuBC, fvBC, fsBC, number=1, checkSolvers=True)

Initializing solver...done.
       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  1.39365e+03  8.93986e+04  2.32804e+02 -5.06391e-08  1.78456e-16  4.65474e-15 
CPU times: user 46.6 s, sys: 2.85 s, total: 49.5 s
Wall time: 2.81 s


In [19]:
%time x, *_ = solver.steps(x1, fuBC, fvBC, fsBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  3.94809e+02  8.88307e+04  2.11836e+00  4.16364e-08  1.96753e-16  6.59807e-15 
       2  3.00000e-02  3.90526e+02  1.24713e+03  1.98458e+00  4.32535e-08  1.96231e-16  6.41150e-15 
       3  4.50000e-02  3.87209e+02  1.20003e+03  1.84210e+00  4.44603e-08  1.95128e-16  5.88998e-15 
       4  6.00000e-02  3.85515e+02  7.64226e+02  1.72136e+00  4.56186e-08  1.96093e-16  4.97246e-15 
       5  7.50000e-02  3.84973e+02  2.80624e+02  1.62183e+00  4.67380e-08  1.96106e-16  4.98764e-15 
       6  9.00000e-02  3.84840e+02  8.37754e+01  1.54098e+00  4.78288e-08  1.94435e-16  5.11110e-15 
       7  1.05000e-01  3.84593e+02  1.40824e+02  1.47506e+00  4.88990e-08  1.95012e-16  4.64027e-15 
       8  1.20000e-01  3.84154e+02  2.64729e+02  1.41987e+00  4.99570e-08  1.95186e-16  4.49672e-15 
       9  1.35000e-01  3.83768e+02  2.55747e+02  1.37224e+00  5.10123e-08  1.94195e-16  4.5